# Coding Discussion 03

## By Sonali Subbu Rathinam - ss4608
## Date : 10/10/2021

## Instructions

## Task

Please read in the Chicago Summer 2018 Crimes Dataset located in the repository folder.

Using the data wrangling methods covered in class this week, create a new data frame where:

- the **_unit of observation_** is the crime type (i.e. `primary_type`),
- the **_column variables_** corresponds with the **_day of the month_**, and
- **_each cell_** is populated by the **_proportion of times that crime type was committed over all days of the month_**
    + For example, assume there were just two days in a month and 2 thefts were committed on the first day, and 1 on the second day, then the _proportion_ of thefts committed on the first day would be .66 and .33 on the second day).

Make sure that:

- all missing values are filled with zeros. Zeros in this case means no crimes were committed that day;
- the data is rounded to the second decimal place; and
- the data frame is printed at the end of the notebook.


In [1]:
#Importing pandas so that it can be used
import pandas as pd

In [2]:
#Reading the csv data into a dataframe called df.
df = pd.read_csv("chicago_summer_2018_crime_data.csv")
df.head()  #To check if the data has been read properly 

,month,day,year,day_of_week,description,location_description,block,primary_type,district,ward,arrest,domestic,latitude,longitude
0,8,4,2018,Saturday,FROM BUILDING,APARTMENT,039XX W WASHINGTON BLVD,THEFT,11,28.0,False,False,NaN,NaN
1,7,26,2018,Thursday,POCKET-PICKING,RESTAURANT,005XX W MADISON ST,THEFT,1,42.0,False,False,NaN,NaN
2,6,24,2018,Sunday,BOGUS CHECK,GROCERY FOOD STORE,004XX E 34TH ST,DECEPTIVE PRACTICE,2,4.0,False,False,NaN,NaN
3,6,13,2018,Wednesday,SIMPLE,RESIDENCE,098XX S EXCHANGE AVE,ASSAULT,4,10.0,False,True,NaN,NaN
4,6,14,2018,Thursday,TO VEHICLE,STREET,001XX S WALLER AVE,CRIMINAL DAMAGE,15,29.0,False,False,NaN,NaN


In [3]:
#Finding the total count of each type of crime in a particular month using vertical method chaining. Filter command gives the
#required columns, which are then grouped. The size function is used to find the observation count, and it is stored in a 
#column named count1.
a = \
(df
 .filter(["primary_type","month"])
 .groupby(['primary_type', 'month'])
 .size() 
 .reset_index()
 .rename(columns={0:"count1"})
)

print(a)

         primary_type  month  count1
0               ARSON      6      36
1               ARSON      7      39
2               ARSON      8      37
3             ASSAULT      6    1872
4             ASSAULT      7    1937
..                ...    ...     ...
86              THEFT      7    6123
87              THEFT      8    6438
88  WEAPONS VIOLATION      6     495
89  WEAPONS VIOLATION      7     546
90  WEAPONS VIOLATION      8     560

[91 rows x 3 columns]


In [4]:
#Finding the total count of each type of crime in a particular day of each month using vertical method chaining. 
#Filter command gives the required columns, which are then grouped. The size function is used to find the observation count, 
#and it is stored in a column named count2.
b = \
(df
 .filter(["primary_type","month","day"])
 .groupby(['primary_type', 'month', 'day'])
 .size() # We can count the number of observations using the size method
 .reset_index()
 .rename(columns={0:"count2"})
)

print(b)

           primary_type  month  day  count2
0                 ARSON      6    1       2
1                 ARSON      6    2       1
2                 ARSON      6    3       2
3                 ARSON      6    4       1
4                 ARSON      6    5       1
...                 ...    ...  ...     ...
2037  WEAPONS VIOLATION      8   27      15
2038  WEAPONS VIOLATION      8   28      13
2039  WEAPONS VIOLATION      8   29      12
2040  WEAPONS VIOLATION      8   30      22
2041  WEAPONS VIOLATION      8   31      23

[2042 rows x 4 columns]


In [5]:
#Merging the two newly created dataframes using left join
c=a.merge(b, how="left", on=["primary_type", "month"])
print(c)

           primary_type  month  count1  day  count2
0                 ARSON      6      36    1       2
1                 ARSON      6      36    2       1
2                 ARSON      6      36    3       2
3                 ARSON      6      36    4       1
4                 ARSON      6      36    5       1
...                 ...    ...     ...  ...     ...
2037  WEAPONS VIOLATION      8     560   27      15
2038  WEAPONS VIOLATION      8     560   28      13
2039  WEAPONS VIOLATION      8     560   29      12
2040  WEAPONS VIOLATION      8     560   30      22
2041  WEAPONS VIOLATION      8     560   31      23

[2042 rows x 5 columns]


In [6]:
#In the merged dataframe, we are creating a column named proportions, that will give us the proportion of times a crime
#was committed over all the days of that particular month. We are also rounding the values to the second decimal place.
c['proportions']=round(c['count2']/c['count1'], 2)

print(c)

           primary_type  month  count1  day  count2  proportions
0                 ARSON      6      36    1       2         0.06
1                 ARSON      6      36    2       1         0.03
2                 ARSON      6      36    3       2         0.06
3                 ARSON      6      36    4       1         0.03
4                 ARSON      6      36    5       1         0.03
...                 ...    ...     ...  ...     ...          ...
2037  WEAPONS VIOLATION      8     560   27      15         0.03
2038  WEAPONS VIOLATION      8     560   28      13         0.02
2039  WEAPONS VIOLATION      8     560   29      12         0.02
2040  WEAPONS VIOLATION      8     560   30      22         0.04
2041  WEAPONS VIOLATION      8     560   31      23         0.04

[2042 rows x 6 columns]


In [7]:
#Using the pivot table function to convert the merged dataframe from long to wide. In this, we are also filling all the 
#missing values to zero.
df_required=c.pivot_table(values='proportions', columns='day', index=['primary_type', 'month'], fill_value=0)

In [8]:
#The dataframe is printed at the end of the notebook
print(df_required)

day                        1     2     3     4     5     6     7     8     9   \
primary_type      month                                                         
ARSON             6      0.06  0.03  0.06  0.03  0.03  0.03  0.06  0.03  0.00   
                  7      0.05  0.00  0.00  0.00  0.03  0.05  0.08  0.03  0.03   
                  8      0.00  0.05  0.03  0.03  0.05  0.08  0.00  0.08  0.03   
ASSAULT           6      0.04  0.04  0.03  0.04  0.03  0.04  0.03  0.03  0.03   
                  7      0.04  0.04  0.03  0.03  0.03  0.04  0.03  0.04  0.03   
...                       ...   ...   ...   ...   ...   ...   ...   ...   ...   
THEFT             7      0.03  0.04  0.03  0.02  0.03  0.03  0.03  0.02  0.03   
                  8      0.03  0.04  0.04  0.04  0.03  0.03  0.04  0.03  0.03   
WEAPONS VIOLATION 6      0.03  0.03  0.03  0.03  0.03  0.03  0.02  0.02  0.04   
                  7      0.03  0.02  0.04  0.06  0.04  0.03  0.04  0.04  0.03   
                  8      0.0